In [8]:
import os
import mmcv
import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [9]:
epoch = 'uniform_model_soup_mysoup'
work_dir = '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k'

cfg = Config.fromfile('/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k.py')
root='/opt/ml/input/data/images/test'

cfg.data.test.img_dir = root
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [10]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2023-01-05 08:10:16,359 - mmseg - INFO - Loaded 819 images


In [11]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/uniform_model_soup_mysoup.pth


In [12]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 3.1 task/s, elapsed: 264s, ETA:     0s

In [13]:
input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/input/code/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# PredictionString
for img_id, pred in enumerate(output):

    img_id = datas["images"][img_id]
    file_name = img_id["file_name"]

    temp_mask = []
    pred = pred.reshape(1, 512, 512)
    mask = pred.reshape((1, output_size, input_size//output_size, output_size, input_size//output_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
    string = oms.flatten()
    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   ,ignore_index=True)


submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)